# Predict Podcast Listening Time

## Modified the metric from `log1p RMSE` to Raw `RMSE` for both Keras and Keras Tuner

## Keras - 1 Attempt

## Attempt 1

In [ ]:

# Set seeds for reproducibility
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import time

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Data loading
train_df = pd.read_csv('playground-series-s5e4/train.csv')
test_df = pd.read_csv('playground-series-s5e4/test.csv')

id_col = 'id'
target_columns = ['Listening_Time_minutes']
competition_problem_subtype = 'continuous-regression'

# Prepare target
Y = train_df[target_columns].astype(float).values
if np.all(Y >= 0):
    y_enc = np.log1p(Y)
else:
    y_enc = Y

X = train_df.drop(columns=target_columns + [id_col], errors='ignore')

# Assign train and validation sets (test split provided)
X_train = X.copy()
y_train = y_enc
train_ids = train_df[id_col]

X_val = test_df.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = test_df[id_col]
y_val = None

# Feature engineering: drop all-missing columns
all_missing = X_train.columns[X_train.isna().all()].tolist()
X_train.drop(columns=all_missing, inplace=True)
X_val.drop(columns=all_missing, inplace=True)

# Identify categorical columns and drop high-cardinality ones
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
high_card = [c for c in cat_cols if X_train[c].nunique() > 50]
X_train.drop(columns=high_card, inplace=True)
X_val.drop(columns=high_card, inplace=True)
cat_cols = [c for c in cat_cols if c not in high_card]
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

# Model architecture selection
dataset_size, feature_count = X_train_proc.shape
if dataset_size < 10000 or feature_count < 100:
    layer_sizes = [min(feature_count * 2, 128), min(feature_count, 64)]
    dropout_rate = 0.3
    use_batchnorm = False
else:
    raw_sizes = [feature_count * i for i in (2, 1, 0.5, 0.25)]
    layer_sizes = [int(s) for s in raw_sizes if s >= 16]
    dropout_rate = 0.4
    use_batchnorm = True

model = Sequential()
for idx, size in enumerate(layer_sizes):
    if idx == 0:
        model.add(Dense(size, activation='relu', input_shape=(feature_count,)))
    else:
        model.add(Dense(size, activation='relu'))
    model.add(Dropout(dropout_rate))
    if use_batchnorm:
        from tensorflow.keras.layers import BatchNormalization
        model.add(BatchNormalization())

# Output layer for regression
model.add(Dense(1, activation='linear'))

def mse_real(y_true_log, y_pred_log):
    y_true = tf.math.expm1(y_true_log)
    y_pred = tf.math.expm1(y_pred_log)
    return tf.reduce_mean(tf.square(y_true - y_pred))
mse_real.__name__ = 'mse_real'      

def rmse_real(y_true_log, y_pred_log):
    y_true = tf.math.expm1(y_true_log)
    y_pred = tf.math.expm1(y_pred_log)
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
rmse_real.__name__ = 'rmse_real'

# Compile
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=[rmse_real,
             mse_real]
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
    verbose=2
)
training_duration = time.time() - start_time

# Logging results
results = {
    'training_loss': history.history['mse_real'][-1],
    'validation_loss': history.history['val_mse_real'][-1],
    'training_RMSE': history.history['rmse_real'][-1],
    'validation_RMSE': history.history['val_rmse_real'][-1]
}
with open('Keras/results.json', 'w') as f:
    json.dump(results, f)

# Prediction on test set
raw_preds = model.predict(X_val_proc)
final_preds = raw_preds
if np.all(final_preds >= 0):
    final_preds = np.expm1(np.clip(final_preds, a_min=None, a_max=20))
final_preds = final_preds.reshape(-1, 1)

# Prepare submission
submission = pd.DataFrame(final_preds, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('Keras/submission_result.csv', index=False)


/home/exouser/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


2025-07-22 04:05:57.880797: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 182400000 exceeds 10% of free system memory.



Epoch 1: val_loss improved from inf to 0.18842, saving model to best_model.h5


18750/18750 - 45s - 2ms/step - loss: 0.2810 - mse_real: 207445041348608.0000 - rmse_real: 105186.5859 - val_loss: 0.1884 - val_mse_real: 185.5134 - val_rmse_real: 13.4039
Epoch 2/100

Epoch 2: val_loss did not improve from 0.18842
18750/18750 - 38s - 2ms/step - loss: 0.1925 - mse_real: 237.6077 - rmse_real: 15.1517 - val_loss: 0.1917 - val_mse_real: 190.9299 - val_rmse_real: 13.5928
Epoch 3/100

Epoch 3: val_loss improved from 0.18842 to 0.18401, saving model to best_model.h5


...

18750/18750 - 31s - 2ms/step - loss: 0.1834 - mse_real: 219.1556 - rmse_real: 14.5397 - val_loss: 0.1716 - val_mse_real: 180.4725 - val_rmse_real: 13.1997
Epoch 18/100

Epoch 18: val_loss did not improve from 0.17163
18750/18750 - 31s - 2ms/step - loss: 0.1834 - mse_real: 219.0659 - rmse_real: 14.5426 - val_loss: 0.1742 - val_mse_real: 181.5730 - val_rmse_real: 13.2385
Epoch 19/100

Epoch 19: val_loss did not improve from 0.17163
18750/18750 - 37s - 2ms/step - loss: 0.1827 - mse_real: 218.2859 - rmse_real: 14.5182 - val_loss: 0.1728 - val_mse_real: 185.7664 - val_rmse_real: 13.3941
Epoch 20/100

Epoch 20: val_loss did not improve from 0.17163
18750/18750 - 38s - 2ms/step - loss: 0.1830 - mse_real: 218.9046 - rmse_real: 14.5372 - val_loss: 0.1760 - val_mse_real: 180.9984 - val_rmse_real: 13.2175
Epoch 21/100

Epoch 21: val_loss improved from 0.17163 to 0.17067, saving model to best_model.h5


18750/18750 - 38s - 2ms/step - loss: 0.1830 - mse_real: 218.9275 - rmse_real: 14.5325 - val_loss: 0.1707 - val_mse_real: 182.5978 - val_rmse_real: 13.2735
Epoch 22/100

Epoch 22: val_loss improved from 0.17067 to 0.16936, saving model to best_model.h5


18750/18750 - 38s - 2ms/step - loss: 0.1835 - mse_real: 219.3958 - rmse_real: 14.5501 - val_loss: 0.1694 - val_mse_real: 181.8522 - val_rmse_real: 13.2472
Epoch 23/100

Epoch 23: val_loss did not improve from 0.16936
18750/18750 - 38s - 2ms/step - loss: 0.1832 - mse_real: 218.8951 - rmse_real: 14.5362 - val_loss: 0.1707 - val_mse_real: 188.1691 - val_rmse_real: 13.4791
Epoch 24/100

Epoch 24: val_loss did not improve from 0.16936
18750/18750 - 34s - 2ms/step - loss: 0.1829 - mse_real: 219.0678 - rmse_real: 14.5448 - val_loss: 0.1724 - val_mse_real: 182.8924 - val_rmse_real: 13.2870
Epoch 25/100

Epoch 25: val_loss did not improve from 0.16936
18750/18750 - 36s - 2ms/step - loss: 0.1831 - mse_real: 220.1538 - rmse_real: 14.5744 - val_loss: 0.1739 - val_mse_real: 183.7322 - val_rmse_real: 13.3203
Epoch 26/100

Epoch 26: val_loss did not improve from 0.16936
18750/18750 - 38s - 2ms/step - loss: 0.1829 - mse_real: 219.3247 - rmse_real: 14.5505 - val_loss: 0.1748 - val_mse_real: 185.4078 - 

2025-07-22 04:25:03.545477: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76000000 exceeds 10% of free system memory.


7813/7813 ━━━━━━━━━━━━━━━━━━━━ 5s 626us/step


## Keras Tuner - 1 Attempt

## Attempt 1


In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import time

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Data loading
train_df = pd.read_csv('playground-series-s5e4/train.csv')
test_df = pd.read_csv('playground-series-s5e4/test.csv')

id_col = 'id'
target_columns = ['Listening_Time_minutes']
competition_problem_subtype = 'continuous-regression'

# Prepare target
Y = train_df[target_columns].astype(float).values
if np.all(Y >= 0):
    y_enc = np.log1p(Y)
else:
    y_enc = Y

X = train_df.drop(columns=target_columns + [id_col], errors='ignore')

# Assign train and validation sets (test split provided)
X_train = X.copy()
y_train = y_enc
train_ids = train_df[id_col]

X_val = test_df.drop(columns=target_columns + [id_col], errors='ignore')
test_ids = test_df[id_col]
y_val = None

# Feature engineering: drop all-missing columns
all_missing = X_train.columns[X_train.isna().all()].tolist()
X_train.drop(columns=all_missing, inplace=True)
X_val.drop(columns=all_missing, inplace=True)

# Identify categorical columns and drop high-cardinality ones
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
high_card = [c for c in cat_cols if X_train[c].nunique() > 50]
X_train.drop(columns=high_card, inplace=True)
X_val.drop(columns=high_card, inplace=True)
cat_cols = [c for c in cat_cols if c not in high_card]
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

# Model architecture selection
dataset_size, feature_count = X_train_proc.shape
if dataset_size < 10000 or feature_count < 100:
    layer_sizes = [min(feature_count * 2, 128), min(feature_count, 64)]
    dropout_rate = 0.3
    use_batchnorm = False
else:
    raw_sizes = [feature_count * i for i in (2, 1, 0.5, 0.25)]
    layer_sizes = [int(s) for s in raw_sizes if s >= 16]
    dropout_rate = 0.4
    use_batchnorm = True

import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Input dimension
n_features = feature_count

def mse_real(y_true_log, y_pred_log):
    y_true = tf.math.expm1(y_true_log)
    y_pred = tf.math.expm1(y_pred_log)
    return tf.reduce_mean(tf.square(y_true - y_pred))
mse_real.__name__ = 'mse_real'      

def rmse_real(y_true_log, y_pred_log):
    y_true = tf.math.expm1(y_true_log)
    y_pred = tf.math.expm1(y_pred_log)
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
rmse_real.__name__ = 'rmse_real'



# HyperModel
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024, step=64)
        drop = hp.Float('dropout', 0.0, 0.5, step=0.1)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        model = Sequential()
        for idx in range(layers):
            if idx == 0:
                model.add(Dense(units, activation='relu', input_shape=(n_features,)))
            else:
                model.add(Dense(units, activation='relu'))
            model.add(Dropout(drop))
            model.add(BatchNormalization())
        model.add(Dense(1, activation='linear'))  # Output layer for regression
        model.compile(optimizer=opt, loss='mean_squared_error', metrics=[rmse_real, mse_real])
        return model

# Tuner
bs = 64  # batch size
ep = 100  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=False,
    project_name='bayesian_tuner'
)

tuner.search(
    X_train_proc, y_train,
    validation_split=0.2,
    batch_size=bs, epochs=ep,
    callbacks=[early_stopping, checkpoint]
)

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain model with original callbacks and data
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100, batch_size=bs,
    callbacks=[early_stopping, checkpoint],
    verbose=2
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
    verbose=2
)
training_duration = time.time() - start_time

# Logging results
results = {
    'training_loss': history.history['mse_real'][-1],
    'validation_loss': history.history['val_mse_real'][-1],
    'training_RMSE': history.history['rmse_real'][-1],
    'validation_RMSE': history.history['val_rmse_real'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction on test set
raw_preds = model.predict(X_val_proc)
final_preds = raw_preds
if np.all(final_preds >= 0):
    final_preds = np.expm1(np.clip(final_preds, a_min=None, a_max=20))
final_preds = final_preds.reshape(-1, 1)

# Prepare submission
submission = pd.DataFrame(final_preds, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 11m 52s]
val_loss: 0.16736340522766113

Best val_loss So Far: 0.16674523055553436
Total elapsed time: 02h 04m 20s
Epoch 1/100


9375/9375 - 116s - 12ms/step - loss: 0.2959 - mse_real: 322.7877 - rmse_real: 17.0280 - val_loss: 0.1785 - val_mse_real: 196.0715 - val_rmse_real: 13.8873
Epoch 2/100


9375/9375 - 133s - 14ms/step - loss: 0.2008 - mse_real: 244.4143 - rmse_real: 15.3926 - val_loss: 0.1706 - val_mse_real: 195.1746 - val_rmse_real: 13.8445
Epoch 3/100
9375/9375 - 125s - 13ms/step - loss: 0.1924 - mse_real: 237.5933 - rmse_real: 15.1860 - val_loss: 0.1791 - val_mse_real: 209.2784 - val_rmse_real: 14.3411
Epoch 4/100
9375/9375 - 131s - 14ms/step - loss: 0.1856 - mse_real: 233.3987 - rmse_real: 15.0595 - val_loss: 0.1734 - val_mse_real: 193.2887 - val_rmse_real: 13.7707
Epoch 5/100
9375/9375 - 126s - 13ms/step - loss: 0.1820 - mse_real: 229.8193 - rmse_real: 14.9472 - val_loss: 0.1728 - val_mse_real: 181.3948 - val_rmse_real: 13.3373
Epoch 6/100
9375/9375 - 125s - 13ms/step - loss: 0.1797 - mse_real: 226.7112 - rmse_real: 14.8476 - val_loss: 0.1712 - val_mse_real: 184.1029 - val_rmse_real: 13.4352
Epoch 7/100


9375/9375 - 121s - 13ms/step - loss: 0.1774 - mse_real: 223.5650 - rmse_real: 14.7450 - val_loss: 0.1677 - val_mse_real: 181.1964 - val_rmse_real: 13.3317
Epoch 8/100


9375/9375 - 100s - 11ms/step - loss: 0.1755 - mse_real: 220.9261 - rmse_real: 14.6598 - val_loss: 0.1673 - val_mse_real: 181.4811 - val_rmse_real: 13.3426
Epoch 9/100
9375/9375 - 101s - 11ms/step - loss: 0.1736 - mse_real: 218.7231 - rmse_real: 14.5896 - val_loss: 0.1688 - val_mse_real: 182.1864 - val_rmse_real: 13.3693
Epoch 10/100
9375/9375 - 103s - 11ms/step - loss: 0.1716 - mse_real: 216.5070 - rmse_real: 14.5167 - val_loss: 0.1710 - val_mse_real: 185.7262 - val_rmse_real: 13.4912
Epoch 11/100
9375/9375 - 100s - 11ms/step - loss: 0.1698 - mse_real: 214.6200 - rmse_real: 14.4535 - val_loss: 0.1720 - val_mse_real: 184.6041 - val_rmse_real: 13.4509
Epoch 12/100
9375/9375 - 98s - 10ms/step - loss: 0.1678 - mse_real: 212.7063 - rmse_real: 14.3903 - val_loss: 0.1714 - val_mse_real: 183.8074 - val_rmse_real: 13.4231
Epoch 13/100
9375/9375 - 98s - 10ms/step - loss: 0.1658 - mse_real: 210.8697 - rmse_real: 14.3278 - val_loss: 0.1739 - val_mse_real: 185.0168 - val_rmse_real: 13.4638
Epoch 14

18750/18750 - 133s - 7ms/step - loss: 0.1904 - mse_real: 247.6974 - rmse_real: 15.3408 - val_loss: 0.1684 - val_mse_real: 182.9453 - val_rmse_real: 13.2702
Epoch 2/100

Epoch 2: val_loss improved from 0.16835 to 0.16819, saving model to best_model.h5


18750/18750 - 133s - 7ms/step - loss: 0.1871 - mse_real: 243.1248 - rmse_real: 15.2032 - val_loss: 0.1682 - val_mse_real: 181.7146 - val_rmse_real: 13.2276
Epoch 3/100

Epoch 3: val_loss did not improve from 0.16819
18750/18750 - 140s - 7ms/step - loss: 0.1844 - mse_real: 239.7473 - rmse_real: 15.0963 - val_loss: 0.1684 - val_mse_real: 183.5396 - val_rmse_real: 13.2857
Epoch 4/100

Epoch 4: val_loss did not improve from 0.16819
18750/18750 - 137s - 7ms/step - loss: 0.1820 - mse_real: 237.2122 - rmse_real: 15.0155 - val_loss: 0.1690 - val_mse_real: 183.3835 - val_rmse_real: 13.2811
Epoch 5/100

Epoch 5: val_loss did not improve from 0.16819
18750/18750 - 138s - 7ms/step - loss: 0.1802 - mse_real: 235.8288 - rmse_real: 14.9694 - val_loss: 0.1701 - val_mse_real: 186.0477 - val_rmse_real: 13.3792
Epoch 6/100

Epoch 6: val_loss did not improve from 0.16819
18750/18750 - 142s - 8ms/step - loss: 0.1786 - mse_real: 234.1857 - rmse_real: 14.9176 - val_loss: 0.1701 - val_mse_real: 186.3042 - val

In [ ]:
print(training_duration)

1468.453455
